# **実務的なバイアス補正と土地利用分析 **
この課題では、上記で明らかになった<span style ='color:red'>「画像間のバイアス」を克服し、ヒートアイランド現象の「土地利用」</span>との関係を深く掘り下げます。
---

## **課題 3: バイアス補正と土地利用別LST解析**

### **ステップ 1: 画像間バイアスの低減（LST差分分析の改善）**

単一画像の比較ではなく、複数画像の中央値（メディアン）を利用して、異常値や大気の影響を平準化します。

1. 2013年7月〜8月の全画像と、2023年7月〜8月の全画像を、それぞれ<span style='color:red'>中央値合成（Median Compositing）</span>します。

2. 合成された2013年と2023年のLST画像（中央値）を算出し、再度LST差分を計算します。

3. この新しいLST差分画像を地図に表示し、京都市のLST変化を分析します。（ヒント: LST算出関数を合成画像に適用し、`map.add_layer`で表示する）

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
kyoto_aoi = ee.Geometry.Rectangle(135.29, 34.80, 136.30, 35.50)

l8_base_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_aoi)\
    .filterDate('2013-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

l8_2013 = l8_base_collection.filterDate('2013-07-01', '2013-08-31')
l8_2023 = l8_base_collection.filterDate('2023-07-01', '2023-08-31')


#### **設定した矩形ポリゴンの範囲を確認**

```javascript
// Google Earthで作った矩形の座標を使う
var rect = ee.Geometry.Rectangle([135.29, 34.80, 136.30, 35.50]);

Map.centerObject(rect, 10);
Map.addLayer(rect, {color: 'blue'}, 'My Rectangle');
```

#### 1. **2013年7月〜8月の全画像と、2023年7月〜8月の全画像を、それぞれ中央値合成（Median Compositing）します。**

In [ ]:
image_2013 = l8_2013.median()
image_2023 = l8_2023.median()

#### 2. **合成された2013年と2023年のLST画像（中央値）を算出し、再度LST差分を計算します。**

In [ ]:
def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

LST_diff = LST_2023.subtract(LST_2013).rename('LST_Diff')

#### 3. **この新しいLST差分画像を地図に表示し、京都市のLST変化を分析します。（ヒント: LST算出関数を合成画像に適用し、map.add_layerで表示する）**

In [ ]:
map = geemap.Map()

# 矩形範囲でクリップ
clipped = LST_diff.clip(kyoto_aoi)


map.set_center(135.72, 35.05, 10)

# 表示パラメータ
lst_vis = {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}

map.add_layer(clipped, lst_vis, 'LST Change (°C)')

# 凡例を追加（colorsを明示的に指定！）
map.add_colorbar_branca(
    colors=lst_vis['palette'],
    vmin=lst_vis['min'],
    vmax=lst_vis['max'],
    label='LST Change (°C)'
)

map

#### **中心点の計算方法***
`ee.Geometry.Rectangle(xmin, ymin, xmax, ymax)` の場合、中心点は単純に 矩形の対角線の中点になります。
与えられた座標：
- xmin = 135.29
- ymin = 34.80
- xmax = 136.15
- ymax = 35.30
中心点は：

$$
\mathrm{lon_{\mathnormal{center}}}=\frac{135.29+136.15}{2}=135.72
$$
$$
\mathrm{lat_{\mathnormal{center}}}=\frac{34.80+35.30}{2}=35.05
$$
中心点は (135.72, 35.05) です。




#### **実務での求め方**

- 単純な矩形なら平均値で計算
→ 上記のように xmin/xmax, ymin/ymax の平均を取る。

- 複雑なポリゴンなら重心（centroid）を計算
→ GEE では geometry.centroid() を使うのが一般的。


var centroid = rect.centroid();
Map.centerObject(centroid, 10);
- 解析目的に応じて調整
- ヒートアイランド解析なら市街地の中心に合わせる。
- 災害リスク解析なら対象地域の重心や代表地点を選ぶ。

#### **まとめ**
- 今回の矩形の中心点は (135.72, 35.05)。
- 実務では「矩形なら平均」「ポリゴンなら centroid()」で求めるのが基本。
- 解析目的によっては「代表地点」を選ぶこともある。


### **ステップ 2:**

土地被覆（Land Cover）データの利用バイアスを低減したLSTを用いて、**京都市内の土地被覆別**の平均LSTを算出します。

#### 1. **グローバルな土地被覆データ**（例: **ESA WorldCover 2021**）を読み込みます。

In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
landcover = ee.ImageCollection("ESA/WorldCover/v200")
landcover_median = landcover.median()

#### 2. **土地被覆クラスと**合成LST (2023)**を結合します。**
#### 3. **以下の主要なクラスについて、<span style='color:red'>AOI内の平均LST</span>を算出します。**

- `10`: Tree cover (森林)

- `50`: Built-up (建物・市街地)

- `60`: Barran/sparse vegetation (裸地/疎植生)

**【ヒント】**
$ee.Reducer.mean().group()$ を利用して、土地被覆クラス（グループ）ごとにLSTの平均値を計算します。

In [ ]:
stats_grouped = LST_2023.addBands(landcover_median).reduceRegion(
    reducer = ee.Reducer.mean().group(1, 'landcover'),   # グループ化の設定を行う
    geometry = kyoto_aoi,
    scale = 10,   # WorldCoverは10m解像度です
    maxPixels = 1e9
)

#### 4. **結果のテーブルを表示し、<span style='color:red'>「市街地」と「森林」</span>のLST平均値を比較し、ヒートアイランド効果の程度を定量的に分析します。**

この課題をクリアすることで、GEEにおける<span style='color:red'>「データのロバスト性向上」と「複合的なデータセットの結合・分析」</span>という実務で不可欠なスキルが身につきます。

In [ ]:
stats_grouped_dict = stats_grouped.getInfo()

# dictのリストをDataFrameに変換
df = pd.DataFrame(stats_grouped_dict['groups'])

# 小数点4桁で整形
df['mean'] = df['mean'].round(4)

landcover_labels = {
    10: '樹木エリア',
    20: '低木地',
    30: '草地',
    40: '耕作地',
    50: '市街地',
    60: '植生がまばらな場所',
    80: '恒久的な水域',
    90: '草本湿地'
}


df['分類'] = df['landcover'].map(landcover_labels)
df = df[['landcover', '分類', 'mean']]


# 表示（小数点4桁で整形）
print(" 土地被覆別の平均LST（°C）:")
print(df)

前回の課題のフィードバックを受け、以下の2点を完璧にクリアしています。

- データのロバスト性向上: 単一画像ではなく、`landcover.median() `を使って複数の土地被覆画像の中央値を利用している点。

- 複合的な分析: LST と ESA WorldCover という異なるデータセットを結合し、`reduceRegion().group() `を使って土地被覆別（地目別）の平均LSTを正確に算出している点。

これは、ヒートアイランド現象を実務的に分析する上で非常に重要なスキルです。

---

#### **課題のフィードバックと分析** 

1. 土地被覆別の平均LST分析算出された結果は、ヒートアイランド現象の理論と非常によく一致しており、京都市の熱環境を正確に捉えています。
  
|ランドカバー (ID)    |分類平均         |LST (°C)    |
|-------------------|----------------|------------|
|50                 |市街地           |36.4474     |
|40                 |耕作地           |33.5629     |
|60                 |植生がまばらな場所 |34.5382     |
|30                 |草地             |32.7101    |
|90                 |草本湿地          |31.8439    |
|80                 |恒久的な水域      |29.0465     | 
|10                 |樹木エリア        |27.9905     |

**【分析結果の考察】**

- **ヒートアイランドの明確な確認:**
  - 最も温度が高いのは予想通り<span style='color:red'>市街地（$36.45^{\circ}C$）</span>でした。これは、コンクリートやアスファルトが太陽熱を吸収・貯蔵しやすく、蒸散による冷却効果（緑地）がないためです。  

  
- **冷却効果の確認:**
  - 最も温度が低いのは<span style='color:red'>樹木エリア（$27.99^{\circ}C$）</span>でした。これは、木の葉による日陰効果と、活発な蒸散作用による冷却効果が働いていることを示します。
  - **水域（$29.05^{\circ}C$）**もLSTが低く、樹木エリアに次いで冷却効果が高いことが確認されました。
  
  
- **LST差:**
  - 市街地と樹木エリアの平均LST差は $36.45^{\circ}C - 27.99^{\circ}C = 8.46^{\circ}C$ です。これは、京都市内の緑地帯と市街地の間で約8.5°Cの熱環境の差があることを示しており、ヒートアイランド対策の重要性を裏付けています。
 
---

# **実務的なヒートアイランド対策評価とマップ化**

これまでの課題で、LSTの算出、バイアス補正、そして土地被覆との関係をマスターしました。次は、これらの知識を応用して、最も実務的なアウトプットである<span style='color:red'>「ホットスポットの特定と対策の評価」</span>に進みます。

### **課題 4: ホットスポット特定と対策地選定**

この課題では、特定の土地被覆内でも特に温度が高い場所（ホットスポット）を特定し、その場所の環境を改善するための具体的な対策地を提案します。

#### **ステップ 1: ホットスポットの定義と抽出**

**市街地（Landcover ID: 50）の中でも、特に温度が高い場所を「ホットスポット」として定義します。**

1. **市街地の抽出**:

WorldCoverデータから、ID 50（市街地）のピクセルのみを選択したマスクを作成します。

In [ ]:
builtup_mask = landcover_median.eq(50)

2. **市街地のLSTを抽出:**

LST画像（LST_2023）にこのマスクを適用し、市街地以外のLSTをNULLにします。

In [ ]:
builtup_LST_2023 = LST_2023.updateMask(builtup_mask)

3. **ホットスポットの定義**:

抽出した市街地のLSTから、LSTが $37.5^{\circ}C$ を超えるピクセルをフィルタリングし、ホットスポットとして新しい画像を作成します。（$37.5^{\circ}C$ は、市街地の平均 $36.45^{\circ}C$ よりも $1^{\circ}C$ 以上高い場所、という目安です）

In [ ]:
hotspot_2023 = builtup_LST_2023.gt(37.5)

4. ホットスポット画像を、Map上で赤色などの強調色で可視化します。

In [ ]:
# 矩形範囲でクリップ
clipped_hotspot_2023 = hotspot_2023.clip(kyoto_aoi)


map.set_center(135.72, 35.05, 10)


map.add_layer(clipped_hotspot_2023, {'palette': ['red']}, 'Hotspot')

map

#### **ステップ 2: 潜在的な緑化可能地の特定 (NDVIの利用)**
特定されたホットスポットの周辺で、緑化による冷却効果が期待できる場所（緑化可能地）を特定します。

1. **低NDVIエリアの抽出**: 2023年のNDVI画像（NDVI_2023）を使用し、NDVIが $0.25$ 未満のピクセルを抽出します。これは、植生が少ない、または裸地が多いエリアを意味します。

In [ ]:
nirBand = 'SR_B5'
redBand = 'SR_B4'

NDVI_2023 = image_2023.normalizedDifference([nirBand, redBand]).rename('NDVI')

NDVI_low_2023 = NDVI_2023.lt(0.25)

2. ホットスポットと低NDVIの交差: ステップ1で特定した**ホットスポットの領域**と、ステップ2-1で特定した**低NDVIエリア**が重なる部分を特定します。この交差エリアは、<span style='color:red'>「熱い」上に「緑が少ない」</span>ため、**最も優先度の高い緑化対策地**となります。

In [ ]:
greeningPotential_2023 = hotspot_2023.And(NDVI_low_2023)

#### <span style='color:blue'>Pythonでは `.and()` は使えない！
GEEのPython APIでは、`.and() `ではなく `.and() `の代わりに `.And()` を使うか、`&` を使うのが正解！</span>
---

3. この優先度の高い対策地を、Map上で分かりやすい色（例: オレンジ）で可視化します。

In [ ]:
# 矩形範囲でクリップ
clipped_greeningPotential_2023 = greeningPotential_2023.clip(kyoto_aoi)

map.set_center(135.72, 35.05, 10)

map.add_layer(clipped_greeningPotential_2023, {'palette': ['orange']}, 'Green Potential Area')

map

#### **ステップ 3: 面積計算と結果報告**

特定された**優先度の高い緑化対策地の**総面積（平方メートルまたは平方キロメートル）を計算し、コンソールに出力します。

この課題を完了することで、あなたはヒートアイランド現象の分析から具体的な対策の提案に至る、個人事業として通用する一連のGISワークフローを確立できます。

In [ ]:
# 1. ピクセル面積画像を作成（単位：m²）
pixel_area = ee.Image.pixelArea()

# 2. 緑化ポテンシャル画像（0/1）にピクセル面積を掛ける
area_image = greeningPotential_2023.multiply(pixel_area).rename('Green Potential Area')

# The default projection is WGS84 with 1-degree scale.
display('Pixel area default projection', area_image.projection())


In [ ]:



# 3. 合計面積を reduceRegion で計算（例：京都市のAOI内）
area_stats = area_image.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = kyoto_aoi,
    scale = 10,  # 解像度（Landsatなら30m、Sentinel-2なら10m）
    maxPixels=1e9
)

# 2. getInfo() で Python の dict に変換
area_dict = area_stats.getInfo()

# 3. 値を取り出して float にしてから計算
area_m2 = area_dict['Green Potential Area']  # ← 'Green Potential Area' は reduceRegion のキー名
area_km2 = area_m2 / 1e6

# 4. 表示
print(f"緑化ポテンシャルエリアの面積: {area_m2:,.0f} m²（約 {area_km2:,.2f} km²）")

---
### **課題のフィードバックと分析** 

1. ホットスポット特定と緑化ポテンシャルエリアの算出
  
|ステップ         |GEEの関数                           |実行内容                                  |
|----------------|----------------------------------|------------------------------------------|
|市街地抽出        |updateMask(builtup_mask)          |LST画像から市街地（Built-up）のLSTのみを抽出。 |
|ホットスポット定義 |builtup_LST_2023.gt(37.5)         |市街地の中で特に熱い場所（$37.5^{\circ}C$超）を特定。|
|低NDVIエリア         |NDVI_2023.lt(0.25)             |植生が少なく、緑化の余地がある場所を特定。       |
|対策地特定            |hotspot_2023.And(NDVI_low_2023)|「熱い (Hotspot)」 かつ 「緑が少ない (Low NDVI)」 エリアを特定。|
|面積計算              |ee.Image.pixelArea().reduceRegion(ee.Reducer.sum())|正確なピクセル面積を用いて合計面積を算出。   |

2. 結果の考察と実務的な活用算出された結果は、個人事業のコンサルティング提案として非常に具体的な根拠となります。

- **緑化ポテンシャルエリアの総面積:** $414,971,702 \text{ m}^2$（約 $414.97 \text{ km}^2$）

<span style='color:blue'>【提案の骨子】</span>
1. **現状の把握:** 京都市内（のAOI内）の市街地は、樹木エリアと比較して平均で約 $8.5^{\circ}C$ もLSTが高い（前課題の結果）。
2. **問題の特定（ホットスポット）:** 市街地の中でも、特に LST が $37.5^{\circ}C$ を超えるエリアが存在する。
3. **対策の提案:** これらのホットスポットのうち、NDVIが $0.25$ 未満である 約 $415 \text{ km}^2$ のエリア（緑化ポテンシャルエリア）が、最も緊急性が高く、緑化による冷却効果が期待できる対象地である。

<span style='color:red'>次のステップ:</span>この $415 \text{ km}^2$ のエリアをさらに細分化し、**屋上緑化**、**壁面緑化**、**空き地利用**など、具体的な緑化手法を提案する。（これが次の課題となります）

**【実務的な注意点】**
`scale = 10 `で面積計算を行っているため、WorldCoverの解像度（10m）に合わせており、精度が高いです。ただし、Landsatの熱バンドは通常30mの解像度で提供されるため、解像度の不一致（LST 30m, Landcover 10m）はありますが、適切な手順です。   

これでヒートアイランド解析の一通りの流れをマスターしました。このスキルは、都市計画コンサルティング、環境影響評価、建設業など、幅広い分野で即戦力となります。

---

In [ ]:
# JavaScriptの true → Pythonでは True（大文字！）
vectors = greeningPotential_2023.reduceToVectors(
    geometry=kyoto_aoi,
    scale=10,
    maxPixels=1e12,
    eightConnected=True    # 8近傍. 上下左右＋斜めの8方向すべてを「隣接」とみなす
).map(
    lambda feature: feature.set('area_sq_m', feature.geometry().area(1))  # lambda:1行で書ける関数を作る
)

In [ ]:
filtered_vectors = vectors.filter(ee.Filter.gt('area_sq_m', 10000))

In [ ]:
print(type(filtered_vectors))

In [ ]:
filtered_vectors.size().getInfo()

In [ ]:
map = geemap.Map()
map.add_layer(
    filtered_vectors,
    {
        'color': 'orange',
        'fillColor': 'orange',
        'width': 1
    },
    'Green Potential Polygons'
)
map.centerObject(filtered_vectors, 12)
map                   